## Election system of Germany - 2025 Federal Election

Germans elect their members of parliament with two votes. The first vote is for a direct candidate, who is required to receive a plurality vote in their electoral district. The second vote is used to elect a party list in each state as established by its respective party caucus. The Bundestag comprises, then, the seats representing each electoral district on the first vote and the seats allocated to maintain proportionality based on the second vote. Common practice is that direct candidates are also placed on the electoral lists at higher rankings as a fall-back in case they do not win their districts.

### First vote
The first vote allows the elector to vote for a direct candidate of their constituency, who applies for a direct mandate in the Bundestag (see illustration above, no. 2). A plurality voting system is used, which means that the candidate who receives more votes than any other candidate gets the mandate. If the vote results in a tie, the lot drawn by the leader of the regional election is decisive. In this case, the votes for the other candidates are invalid. The primary function of the first vote is to personalize the election. As there are 299 constituencies (regions) at the moment, the same number of mandates in the Bundestag are distributed to the elected candidates in each district. However, the first vote does not determine the power of the parties in the Bundestag. For each direct mandate in a Bundesland the party always receives one mandate fewer from the second vote.

### Second vote
For the distribution of seats in the German Bundestag, the second vote is more important than the first vote. This second vote allows the elector to vote for a party whose candidates are put together on the regional electoral list. Based on the proportion of second votes, the 598 mandates are distributed to the parties which have achieved at least 5 percent of valid second votes.

This will download folder containing two csv files and four pdf files.
First csv file `btw25_wbz_ergebnisse.csv` contains results of the election by voting districts.
Second csv file `btw25_wbz_leitband.csv` contains metadata about the voting districts.

Contents:
The files `btw25_wbz_ergebnisse.csv` contain data records for all approximately 90,000 electoral districts with the following information: 
• Constituency number, municipality code, municipality name, electoral district number, and district type
• Total number of eligible voters, with and without voting card notation
• Total number of voters and voters with voting cards
• Total number of invalid and valid first and second votes
• Number of first and second votes for each party. 
 
An additional file contains the following explanatory information 
(mainly on the territorial status as of December 31, 2024): 
• State names 
• Administrative district names 
• County names 
• Association municipality names 
• Municipality names for the municipality codes listed. 

The municipal codes used are identical to the key numbers in the official 
municipal directory. 

A special feature is the joint postal voting districts for several municipalities. All municipalities 
in a district that form a joint postal voting committee are assigned the same 2-digit number in the additional field EF8 
“Postal voting affiliation.” The municipality code for the joint 
postal voting districts consists of the number “9” and this 2-digit number. The 4-digit 
association municipality code for this district corresponds to the association municipality code if all municipalities belong to the same association.
 Otherwise, it consists of the digits 
“11” and the 2-digit postal vote affiliation number. 


Description of features:
- Wahlkreis: Constituency number (voting district) - there are 299 constituencies in total, Each constituency votes for one direct candidate (first vote) and contributes to the proportional representation of parties (second vote).
- Land: State
- Regierungsbezirk: Administrative district
- Kreis: County
- Verbandsgemeinde: Association municipality
- Gemeinde: Municipality id (city/town)
- Kennziffer Briefwahlzugehörigkeit: Postal ballot code
- Gemeindename: Municipality name (city/town)
- Wahlbezirk: Electoral district number (within the municipality)
- Bezirkstart: District type 
    - 0: Ballot box district
    - 5: Postal voting district
    - 6: Special district
    - 8: District for eligible voters without further details
- Wahlberechtigte (A): Total number of eligible voters
- Wahlberechtigte ohne Sperrvermerk (A1): Total number of eligible voters without voting card notation (i.e., those who can vote in person)
- Wahlberechtigte mit Sperrvermerk (A2): Total number of eligible voters with voting card notation (i.e., those who can only vote by mail)
- Wahlberechtigte nach § 25 Abs. 2 BWO (A3): Total number of eligible voters according to § 25 Abs. 2 BWO (e.g., for certain special districts)
- Wählende (B): Total number of voters
- Wählende mit Wahlschein (B1): Total number of voters including those with voting cards
- Any Erststimmen: First votes for each party
- Any Zweitstimmen: Second votes for each party
- Ungekürzte Wahlbezirksbezeichnung: Unabridged electoral district description
- Bezeichnung des Wahlbezirkes gemäß Anlage 30 zur BWO: Electoral district description according to Annex 30 of the BWO


The unique identifier of a voting district is a combination of `Wahlkreis` (constituency number), `Kreis` (county), and `Gemeinde` (municipality).

In [ ]:
import pandas as pd

dtype_mappings = {
    "Land": str,
    "Regierungsbezirk": str,
    "Kreis": str,
    "Verbandsgemeinde": str,
    "Gemeinde": str,
}
df_elec = pd.read_csv(
    "../../data/raw/election_2025/btw25_wbz_ergebnisse.csv", sep=";", skiprows=4, dtype=dtype_mappings
)

# Add municipality codes from Land, Regierungsbezirk, Kreis, and Gemeinde columns as strings
df_elec["municipality_code"] = (
    df_elec["Land"]
    + df_elec["Regierungsbezirk"]
    + df_elec["Kreis"]
    + df_elec["Verbandsgemeinde"].str.zfill(4)
    + df_elec["Gemeinde"].str.zfill(3)
)

df_elec.head(10)

In [ ]:
from geoscore_de.data_flow.municipality import load_municipality_data

df_muni = load_municipality_data("../../data/raw/municipalities_2022.csv")

In [ ]:
print("Number of unique municipalities in election data:", df_elec["municipality_code"].nunique())
print("Number of unique municipalities in municipality data:", df_muni["MU_ID"].nunique())

In [ ]:
# join data and check for missing municipalities
df_merged = df_elec.merge(df_muni, left_on="municipality_code", right_on="MU_ID", how="outer", indicator=True)
df_merged["_merge"].value_counts()

## Explore with municipalities are missing in Election data

In [ ]:
df_merged[df_merged["_merge"] == "right_only"][["MU_ID", "Municipality"]]

## Explore missing municipalities in municipality data

In [ ]:
df_merged[df_merged["_merge"] == "left_only"][["municipality_code", "Gemeindename"]].drop_duplicates()